In [1]:
import json
from nltk.corpus import stopwords
import re
import pymorphy2
import gensim.corpora as corpora
import pyLDAvis.gensim_models
import pyLDAvis
import pickle
import os


morph = pymorphy2.MorphAnalyzer()

/home/alexanderlakiza/.virtualenvs/cs224/lib/python3.8/site-packages/pymorphy2/units/base.py:70: DeprecationWarning: inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspec()
  args, varargs, kw, default = inspect.getargspec(cls.__init__)
/home/alexanderlakiza/.virtualenvs/cs224/lib/python3.8/site-packages/pymorphy2/units/base.py:70: DeprecationWarning: inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspec()
  args, varargs, kw, default = inspect.getargspec(cls.__init__)
/home/alexanderlakiza/.virtualenvs/cs224/lib/python3.8/site-packages/pymorphy2/units/base.py:70: DeprecationWarning: inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspec()
  args, varargs, kw, default = inspect.getargspec(cls.__init__)
/home/alexanderlakiza/.virtualenvs/cs224/lib/python3.8/site-packages/pymorphy2/units/base.py:70: DeprecationWarning: inspect.getargspec() is d

### Предобработка

Подгружаем словарь, где ключи - заголовки документов, ключи - токены этого документа в начальной форме

In [2]:
with open("../data/corpus_as_dict_of_norms.json") as f:
    normed_dict = json.load(f)

In [3]:
corpus = [normed_dict[i] for i in normed_dict]

In [4]:
def del_stops(lst):
    """
    Функция удаления стоп-слов
    """
    for word in lst:
        word = re.sub("[^а-яА-ЯёЁ]"," ", re.sub(r'\((.*?)\)', "", word))
    lst = [word for word in lst if not re.match(r'[a-zA-Z]+', word)]  # Удаление английских слов
    lst = [word for word in lst if morph.parse(word)[0].tag.POS not in ['CONJ', 'PRCL', 'INTJ', 'PREP', 'NPRO', 'NUMR']]
    # Удаление служебных частей речи и личных местоимений
    stops = stopwords.words("english") + stopwords.words("russian") + ["это", "который", "наш", "мочь", "год", 
                                            "такой", "знать", "мы", "свой", "один", "другой", "хотеть",
                                            "человек", "всё", "все", "весь", "очень", "думать", "нужно",
                                            "большой", "время", "использовать", "говорить", "сказать",
                                            "иметь", "сделать", "первый", "каждый", "день", "её", "ваш",
                                            "стать", "больший", "ваше", "день", "самый", "понять",
                                            "просто", "ещё", "проблема", "также", "например", "м", "c"]
    return [w for w in lst if w not in stops]

In [5]:
for i in range(len(corpus)):
    corpus[i] = del_stops(corpus[i])

In [6]:
all_words = []
for doc in corpus:
    all_words.extend(doc)

In [7]:
len(all_words)  # Интересно, что всего ровно 24 тысячи слов

24000

### Работа с LDA

In [8]:
id2word = corpora.Dictionary(corpus)

In [9]:
docs = corpus
corpus = [id2word.doc2bow(doc) for doc in docs]

In [10]:
print(corpus[0])

[(0, 3), (1, 4), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 3), (8, 4), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 2), (20, 1), (21, 1), (22, 1), (23, 2), (24, 2), (25, 1), (26, 7), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 2), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 2), (43, 1), (44, 2), (45, 1), (46, 1), (47, 2), (48, 1), (49, 1), (50, 1), (51, 1), (52, 2), (53, 1), (54, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1), (62, 1), (63, 2), (64, 1), (65, 1), (66, 1), (67, 1), (68, 1), (69, 1), (70, 1), (71, 2), (72, 1), (73, 1), (74, 1), (75, 1), (76, 1), (77, 1), (78, 1), (79, 1), (80, 1), (81, 1)]


In [11]:
from gensim.models import LdaMulticore
lda_model = LdaMulticore(corpus=corpus, id2word=id2word, num_topics=7)

In [12]:
lda_model.print_topics()

[(0,
  '0.009*"судный" + 0.008*"являться" + 0.006*"северный" + 0.004*"судно" + 0.003*"библиотека" + 0.003*"вода" + 0.003*"европа" + 0.003*"морской" + 0.003*"страна" + 0.003*"течение"'),
 (1,
  '0.007*"являться" + 0.005*"библиотека" + 0.004*"электрический" + 0.003*"язык" + 0.003*"крупный" + 0.003*"северный" + 0.003*"мир" + 0.003*"город" + 0.003*"век" + 0.003*"часть"'),
 (2,
  '0.008*"город" + 0.006*"являться" + 0.004*"длина" + 0.004*"крупный" + 0.004*"население" + 0.004*"судный" + 0.003*"центр" + 0.003*"вид" + 0.003*"автобус" + 0.003*"часть"'),
 (3,
  '0.006*"вид" + 0.006*"являться" + 0.005*"город" + 0.004*"штат" + 0.004*"сша" + 0.003*"титаник" + 0.003*"население" + 0.003*"мир" + 0.003*"называть" + 0.003*"часть"'),
 (4,
  '0.006*"вид" + 0.005*"являться" + 0.005*"вода" + 0.004*"продукт" + 0.004*"мясо" + 0.004*"использоваться" + 0.004*"судный" + 0.003*"судно" + 0.003*"часть" + 0.003*"называть"'),
 (5,
  '0.005*"являться" + 0.005*"судный" + 0.004*"век" + 0.003*"часть" + 0.003*"вид" + 0.003

In [13]:
pyLDAvis.enable_notebook()
LDAvis_prepared = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
pyLDAvis.save_html(LDAvis_prepared, "../data/LDA.html")

In [14]:
pyLDAvis.display(LDAvis_prepared)

In [15]:
LDAvis_prepared.topic_order

[1, 7, 5, 4, 2, 3, 6]

### Вывод
Сделать выводы по LDA в данном случае крайне сложно. Я пытался брать разное количество тем, но всё равно каждый раз в одной и той же теме встречаются слова, которые не имеют друг к другу никакого отношения, и саму тематику этих слов определить не представляется возможность.
По итогу я выбрал 7 тем, так как это минимальное кол-во тем, при которых круги на графике не пересекаются. Надеюсь, в этом есть какой-то смысл)))
В общем и целом, видно, что во всех "темах" преобладают слова морской тематики, что логично, так как мой корпус собран из статей с википедии, которые упоминались в статье про Титаник